In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from subprocess import check_output, check_call
from glob import glob

import machine
import atlas

from config import paths

In [14]:
building_cases = machine.building_jobids()
len(building_cases)

0

In [15]:
cdr_forcing = "DOR"
vintage = "001"
phase = "deploy"
clobber = False

calc = atlas.global_irf_map(cdr_forcing, vintage)

In [16]:
%%time
calc._refresh_case_status()        
df = calc.df_case_status
df

CPU times: user 836 ms, sys: 1.38 s, total: 2.22 s
Wall time: 2min 26s


,build,submitted,run_completed,archive,error,error_count,timestamp_run,JobId,JobState,Queued,yr_per_day,pe-hr_per_yr
case,,,,,,,,,,,,
smyle.cdr-atlas-v0.control.001,True,True,True,True,False,0,2024-11-04 23:44:49,None,None,False,45.61,471.47
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001,True,True,True,True,False,0,2024-11-05 03:13:08,None,None,False,58.82,365.62
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-04-01_00001.001,True,True,True,True,False,0,2024-11-05 03:12:00,None,None,False,59.00,364.49
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-07-01_00002.001,True,True,True,True,False,0,2024-11-04 20:27:38,None,None,False,58.89,365.17
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-10-01_00003.001,True,True,True,True,False,0,2024-11-04 21:37:34,None,None,False,58.94,364.83
...,...,...,...,...,...,...,...,...,...,...,...,...
smyle.cdr-atlas-v0.glb-dor_Southern_Ocean_038_1999-10-01_02755.001,True,True,True,True,False,0,2025-03-13 04:44:19,None,None,False,55.20,389.56
smyle.cdr-atlas-v0.glb-dor_Southern_Ocean_039_1999-01-01_02756.001,True,True,True,True,False,0,2025-03-13 04:40:44,None,None,False,55.71,385.98
smyle.cdr-atlas-v0.glb-dor_Southern_Ocean_039_1999-04-01_02757.001,True,True,True,True,False,0,2025-03-13 04:39:37,None,None,False,55.95,384.32


In [17]:
sum(df.build.to_list()) # 2761

2761

In [18]:
sum((df.build & ~df.archive).to_list()) # 572

0

In [19]:
sum(df.archive.to_list()) # 2760

2761

In [20]:
sum(df.Queued.to_list())

0

In [21]:
clobber_list = calc.check_cases()
len(clobber_list)

0

In [22]:
n = 0
for case in clobber_list:
    log_files = sorted(glob(f"{paths['scratch']}/{case}/run/cesm.log.????????.??????-??????"))
    if not log_files: 
        print(f"{case}: no log")
        continue

    log_file = log_files[-1]
    n += 1
        
    with open(log_file, "r") as fid:
        lines = fid.readlines()
    CANCELLED = any(["CANCELLED" in l for l in lines])
    print(f"{case}: CANCELLED: {CANCELLED}")
    